Part I: A photo booth application

Recreate the OpenCV logo using OpenCV drawing functions only (e.g., cv2.circle, cv2.line, etc.).

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Create a white background
image = np.ones((500, 500, 3), dtype=np.uint8) * 255

# Draw the three ellipses (circles with part missing for the look)
cv2.ellipse(image, (250, 170), (60, 60), 0, 130, 410, (0, 0, 255), -1)    # Red 
cv2.ellipse(image, (180, 280), (60, 60), 0, 0, 300, (0, 255, 0), -1)   # Green 
cv2.ellipse(image, (320, 280), (60, 60), 0, -60, 240, (255, 0, 0), -1)   # Blue

# Add inner white circles 
cv2.circle(image, (250, 170), 25, (255, 255, 255), -1)
cv2.circle(image, (180, 280), 25, (255, 255, 255), -1)
cv2.circle(image, (320, 280), 25, (255, 255, 255), -1)

# Add the text "OpenCV"
cv2.putText(image, "OpenCV", (160, 380), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 0), 4)

# Convert to RGB for display
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Show using matplotlib
plt.imshow(image_rgb)
plt.axis("off")
plt.show()


Part II: Image Arithmetic

Write a Python script to manually blend two images using NumPy operations.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load your two images
img1 = cv2.imread("image1.jpg")
img2 = cv2.imread("image2.jpg")

# Resize them to the same size if needed
height = min(img1.shape[0], img2.shape[0])
width = min(img1.shape[1], img2.shape[1])
img1 = cv2.resize(img1, (width, height))
img2 = cv2.resize(img2, (width, height))

# Alpha value between 0 and 1
alpha = 0.5

# Manual blend using numpy
blend = (1 - alpha) * img1 + alpha * img2
blend = blend.astype(np.uint8)

# Save the result
cv2.imwrite("manual_blend.jpg", blend)

# Display the result using matplotlib
blend_rgb = cv2.cvtColor(blend, cv2.COLOR_BGR2RGB)
plt.imshow(blend_rgb)
plt.axis("off")
plt.show()


Part IV:

Academic integrity Policy

I, Lance Curio, declare that I have read and understood the Academic Integrity Policy.